<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [5]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

C:\Users\User\AppData\Local\Programs\Python\Python310\python.exe: Error while finding module specification for 'https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1' (ModuleNotFoundError: No module named 'https://zenodo')


In [ ]:
import urllib.request
url = 'https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1'
filename = 'SO_vectors_200.bin'
urllib.request.urlretrieve(url, filename)

In [2]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin", binary=True)

Посмотрим на примере одного слова, что из себя представляет embedding

In [3]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [4]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место? 

Нет,но cats занимает 4-ое

In [6]:
# method most_simmilar
'''your code'''
wv_embeddings.most_similar(positive=['dog'])

[('animal', 0.8564180135726929),
 ('dogs', 0.7880866527557373),
 ('mammal', 0.7623804211616516),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665),
 ('feline', 0.7392398715019226),
 ('bird', 0.7315488457679749),
 ('animal1', 0.7219215631484985),
 ('doggy', 0.7213349342346191),
 ('labrador', 0.7209131717681885)]

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [7]:
import numpy as np
import re
from nltk.tokenize import wordpunct_tokenize
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return wordpunct_tokenize(text)
    
tokenizer = MyTokenizer()

In [8]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    mean_embeddings = np.zeros(shape=dim)
    tokens = tokenizer.tokenize(question)
    n = 0
    for token in tokens:
        if token in embeddings:
            mean_embeddings += np.array(embeddings[token][0:dim])
            n += 1
    if n:
        return mean_embeddings / n        
    
    return mean_embeddings

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [9]:
question_to_vec("I love neural networks", wv_embeddings ,tokenizer) 

array([-1.01422749, -1.68912624, -1.28541217, -1.37103013,  0.15916634,
        0.0670696 ,  1.11884137, -0.07051762,  0.53863693, -0.37139114,
        0.28913016,  1.03125   , -2.86546615, -1.31928726,  2.20982948,
        1.16786722,  1.08419518,  1.84672268, -0.98733809,  0.87209455,
       -0.17219625,  0.73839017, -0.78278264, -0.88278695, -0.61526108,
       -0.15872356, -0.61707981,  0.36668165,  2.01375335,  0.26274722,
       -1.4950124 , -1.38671508, -1.20499369, -1.01661696,  2.76211715,
        1.72557124, -0.19038178, -0.58994474, -0.1141155 , -1.88903529,
       -1.78835048, -0.20420383, -0.70620341,  2.4890339 , -0.15452717,
        1.12550129,  0.26108352,  1.31532462, -0.57667526, -0.33271411,
        1.67649019,  0.91229995, -0.6988014 ,  2.37525521, -1.76342229,
        3.13780514,  1.22432033, -1.05544141,  0.78975705,  0.04404024,
        1.16230452, -0.44780231, -0.9149816 ,  0.01501505, -0.85523087,
       -1.01834532, -0.71621163,  1.0727205 ,  0.88818762, -0.29

In [10]:
res = question_to_vec("I love neural networks", wv_embeddings ,tokenizer)[2]
print(f"Третья(с индексом 2) компонента вектора предложения {res:.2f}")

Третья(с индексом 2) компонента вектора предложения -1.29


### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

Максимум существет при $rank\_q_i^{'}$ >= 2 и $rank\_q_i^{'}$ <= 47 и равен 1

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

DCG@10: 0.3


### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [11]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    '''your code'''
    hits_value = sum(1 for rank in dup_ranks if rank <= k)
    return hits_value / len(dup_ranks)  


In [12]:
import math
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    '''your code'''
    dcg_value = sum(1 / math.log2(rank + 1) for rank in dup_ranks if rank <= k)
    return dcg_value / len(dup_ranks)  


Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [13]:
import pandas as pd

In [14]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [15]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [14]:
!unzip stackoverflow_similar_questions.zip

unzip:  cannot find or open stackoverflow_similar_questions.zip, stackoverflow_similar_questions.zip.zip or stackoverflow_similar_questions.zip.ZIP.


Считайте данные.

In [16]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split("\t"))
    
    return data

Нам понадобиться только файл validation.

In [17]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [18]:
len(validation_data)

3760

Размер нескольких первых строк

In [19]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [21]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    que_vec = [question_to_vec(question, embeddings, tokenizer, dim)] 
    candidate_vecs = [question_to_vec(candidate, embeddings, tokenizer, dim) for candidate in candidates]
    cos_sim = pd.Series(cosine_similarity(X=que_vec, Y=candidate_vecs).squeeze()).sort_values(ascending=False)
    return [(i, candidates[i]) for i in cos_sim.index]


Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [22]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [23]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [21]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

SyntaxError: invalid syntax (1724534416.py, line 5)

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

##### 102 

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [24]:
from tqdm.notebook import tqdm

In [25]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == 0:
        continue
    if i == max_validation_examples:
        break
    q, *ex = line
    ar = [question_to_vec(candidate, wv_embeddings, tokenizer, 200) for candidate in ex]

    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [26]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.276 | Hits@   1: 0.276
DCG@   5: 0.332 | Hits@   5: 0.385
DCG@  10: 0.349 | Hits@  10: 0.437
DCG@ 100: 0.395 | Hits@ 100: 0.671
DCG@ 500: 0.421 | Hits@ 500: 0.874
DCG@1000: 0.434 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [27]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [28]:
words = [list(tokenizer.tokenize(" ".join(row))) for row in train_data]

#### Подсчитаем среднее количество слов в парах после склейки (пригодится для выбора длинны окна)

In [29]:
average_len_pairs = 0
for word in tqdm(words):
    average_len_pairs += len(word)
average_len_pairs = average_len_pairs / len(words)

print(average_len_pairs)

  0%|          | 0/1000000 [00:00<?, ?it/s]

22.424825


#### Прелагаю для начала взять длину окна в 5 а потом сравнить с длинной окна чуть меньшего чем среднее количество слов в склеенных парах (22)

In [30]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                 vector_size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=5).wv

In [31]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [32]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.251 | Hits@   1: 0.251
DCG@   5: 0.320 | Hits@   5: 0.383
DCG@  10: 0.340 | Hits@  10: 0.443
DCG@ 100: 0.395 | Hits@ 100: 0.718
DCG@ 500: 0.422 | Hits@ 500: 0.929
DCG@1000: 0.430 | Hits@1000: 1.000


In [33]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                 vector_size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=19).wv

wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.291 | Hits@   1: 0.291
DCG@   5: 0.356 | Hits@   5: 0.413
DCG@  10: 0.379 | Hits@  10: 0.486
DCG@ 100: 0.429 | Hits@ 100: 0.734
DCG@ 500: 0.455 | Hits@ 500: 0.931
DCG@1000: 0.462 | Hits@1000: 1.000


#### Лучшее качество получается при длинне близкой к среднему количеству слов в парах, потому я остановил свой выбор на 19

#### Проведем эксперементы с стеммерами и лемматизацией

In [38]:
class MyTokenizer:
    def __init__(self, do_stopwords=False, do_lemmatizer=False, do_stemmer=False):
        self.do_stopwords = do_stopwords
        self.do_lemmatizer = do_lemmatizer
        self.do_stemmer = do_stemmer
        if do_stopwords:
            import nltk
            nltk.download('stopwords')
        
        
    def tokenize(self, text): 
        import nltk
        if self.do_lemmatizer:
            lemmatizer = nltk.WordNetLemmatizer()
            if self.do_stopwords:
                from nltk.corpus import stopwords
                stopWords = set(stopwords.words('english'))
                return [lemmatizer.lemmatize(word) for word in nltk.tokenize.wordpunct_tokenize(text) if word not in stopWords]
            else:
                 return [lemmatizer.lemmatize(word) for word in nltk.tokenize.wordpunct_tokenize(text)]
                
        elif self.do_stemmer:
            from nltk.stem import PorterStemmer
            stemmer = PorterStemmer()
            if self.do_stopwords:
                from nltk.corpus import stopwords
                stopWords = set(stopwords.words('english'))
                return [stemmer.stem(word) for word in nltk.tokenize.wordpunct_tokenize(text) if word not in stopWords]
            else:
                 return [stemmer.stem(word) for word in nltk.tokenize.wordpunct_tokenize(text)]
        else:
            if self.do_stopwords:
                from nltk.corpus import stopwords
                stopWords = set(stopwords.words('english'))
                return [word for word in nltk.tokenize.wordpunct_tokenize(text) if word not in stopWords]
            else:
                 return nltk.tokenize.wordpunct_tokenize(text)
    


In [59]:
tokenizer = MyTokenizer(do_stopwords=True, do_lemmatizer=False, do_stemmer=False)
words = [list(tokenizer.tokenize(" ".join(row))) for row in train_data]
print("Data ready")
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                 vector_size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=19).wv
print("Word2Vec ready")
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Data ready
Word2Vec ready


  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.341 | Hits@   1: 0.341
DCG@   5: 0.431 | Hits@   5: 0.509
DCG@  10: 0.457 | Hits@  10: 0.589
DCG@ 100: 0.506 | Hits@ 100: 0.827
DCG@ 500: 0.523 | Hits@ 500: 0.958
DCG@1000: 0.527 | Hits@1000: 1.000


Качество с удалением стопслов выросло

In [39]:
tokenizer = MyTokenizer(do_stopwords=True, do_lemmatizer=True, do_stemmer=False)
words = [list(tokenizer.tokenize(" ".join(row))) for row in train_data]
print("Data ready")
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                 vector_size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=19).wv
print("Word2Vec ready")
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Data ready
Word2Vec ready


  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.338 | Hits@   1: 0.338
DCG@   5: 0.437 | Hits@   5: 0.524
DCG@  10: 0.459 | Hits@  10: 0.593
DCG@ 100: 0.509 | Hits@ 100: 0.834
DCG@ 500: 0.525 | Hits@ 500: 0.958
DCG@1000: 0.529 | Hits@1000: 1.000


In [40]:
tokenizer = MyTokenizer(do_stopwords=True, do_lemmatizer=False, do_stemmer=True)
words = [list(tokenizer.tokenize(" ".join(row))) for row in train_data]
print("Data ready")
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                 vector_size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=19).wv
print("Word2Vec ready")
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Data ready
Word2Vec ready


  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.423 | Hits@   1: 0.423
DCG@   5: 0.504 | Hits@   5: 0.576
DCG@  10: 0.529 | Hits@  10: 0.652
DCG@ 100: 0.576 | Hits@ 100: 0.874
DCG@ 500: 0.589 | Hits@ 500: 0.972
DCG@1000: 0.592 | Hits@1000: 1.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


Лучшее решение было получено при помощи удаления стопслов и стемминга и при помощи эмбедингов обученных на скленных парах вопросов, так происходит потому что удаление стоп слов при токенизации на уровне слов позволяет отбросить
символы что не несут дополнительного смысла для обучения новых эмбедингов для токенизации на уровне слов и для такого небольшого обьема данных, я полагаю что стемминг помог поскольку позволяет привести возможные вариции
форм терминов к одному виду и более корректно учитывать их при построении модели.

В целом плохое качество решения задачи обусловлено небольшим обьемом данных и простой моделью, для решения данной задачи я бы попробовал взять преобученную нейронную сеть дообучить ее на наборах пар и далее использовать ее эмбединги 